In [3]:
import scipy.stats as stats
import math
from yahooquery import Ticker
import pandas as pd


def black_scholes(S, K, T, r, sigma):
    d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    call = S * stats.norm.cdf(d1) - K * math.exp(-r * T) * stats.norm.cdf(d2)
    put = K * math.exp(-r * T) * stats.norm.cdf(-d2) - S * stats.norm.cdf(-d1)
    return call, put


In [4]:
stk = Ticker("AAPL")

In [5]:
option_chain = stk.option_chain
options_df = option_chain.reset_index()
calls_df = options_df[options_df['optionType'] == 'calls']
puts_df = options_df[options_df['optionType'] == 'puts']

calls_df.head()



,symbol,expiration,optionType,contractSymbol,strike,currency,lastPrice,change,percentChange,volume,openInterest,bid,ask,contractSize,lastTradeDate,impliedVolatility,inTheMoney
0,AAPL,2024-11-29,calls,AAPL241129C00100000,100.0,USD,128.35,0.000000,0.000000,1.0,2.0,129.20,130.60,REGULAR,2024-11-21 20:48:15,2.125005,True
1,AAPL,2024-11-29,calls,AAPL241129C00115000,115.0,USD,111.68,0.000000,0.000000,2.0,2.0,114.15,115.65,REGULAR,2024-11-07 16:47:03,1.781251,True
2,AAPL,2024-11-29,calls,AAPL241129C00130000,130.0,USD,100.00,0.000000,0.000000,3.0,3.0,99.20,100.60,REGULAR,2024-11-22 19:07:38,1.484378,True
3,AAPL,2024-11-29,calls,AAPL241129C00135000,135.0,USD,94.70,-5.000000,-5.015045,6.0,5.0,94.15,95.65,REGULAR,2024-11-22 16:28:23,1.390628,True
4,AAPL,2024-11-29,calls,AAPL241129C00140000,140.0,USD,89.38,0.669998,0.755268,3.0,3.0,89.25,90.60,REGULAR,2024-11-22 15:35:41,1.390628,True


In [6]:
calls_df['mid_price'] = (calls_df['ask'] + calls_df['bid']) / 2
calls_df = calls_df[calls_df['mid_price'] > 1]
calls_df.head()

/var/folders/s1/9bykr2ps10l_lqbqpvjd6ntr0000gn/T/ipykernel_6632/163983822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calls_df['mid_price'] = (calls_df['ask'] + calls_df['bid']) / 2


,symbol,expiration,optionType,contractSymbol,strike,currency,lastPrice,change,percentChange,volume,openInterest,bid,ask,contractSize,lastTradeDate,impliedVolatility,inTheMoney,mid_price
0,AAPL,2024-11-29,calls,AAPL241129C00100000,100.0,USD,128.35,0.000000,0.000000,1.0,2.0,129.20,130.60,REGULAR,2024-11-21 20:48:15,2.125005,True,129.900
1,AAPL,2024-11-29,calls,AAPL241129C00115000,115.0,USD,111.68,0.000000,0.000000,2.0,2.0,114.15,115.65,REGULAR,2024-11-07 16:47:03,1.781251,True,114.900
2,AAPL,2024-11-29,calls,AAPL241129C00130000,130.0,USD,100.00,0.000000,0.000000,3.0,3.0,99.20,100.60,REGULAR,2024-11-22 19:07:38,1.484378,True,99.900
3,AAPL,2024-11-29,calls,AAPL241129C00135000,135.0,USD,94.70,-5.000000,-5.015045,6.0,5.0,94.15,95.65,REGULAR,2024-11-22 16:28:23,1.390628,True,94.900
4,AAPL,2024-11-29,calls,AAPL241129C00140000,140.0,USD,89.38,0.669998,0.755268,3.0,3.0,89.25,90.60,REGULAR,2024-11-22 15:35:41,1.390628,True,89.925


In [7]:
# Get current stock price
current_price = stk.price['AAPL']['regularMarketPrice']

# Calculate days to expiration and convert to years
calls_df['days_to_expiry'] = pd.to_datetime(calls_df['expiration']) - pd.Timestamp.now()
calls_df['T'] = calls_df['days_to_expiry'].dt.days / 365

# Use 10-year Treasury rate as risk-free rate (approximate)
r = 0.045

# Calculate theoretical prices using Black-Scholes
calls_df['bs_price'] = calls_df.apply(
    lambda row: black_scholes(
        S=current_price,
        K=row['strike'], 
        T=row['T'],
        r=r,
        sigma=row['impliedVolatility']
    )[0],  # Index 0 gets call price
    axis=1
)

# Compare market vs model prices
calls_df['price_diff'] = calls_df['mid_price'] - calls_df['bs_price']

# Display results
print("Black-Scholes vs Market Prices:")
print(calls_df[['strike', 'mid_price', 'bs_price', 'price_diff']].head())


Black-Scholes vs Market Prices:
   strike  mid_price    bs_price  price_diff
0   100.0    129.900  129.956692   -0.056692
1   115.0    114.900  114.967457   -0.067457
2   130.0     99.900   99.978700   -0.078700
3   135.0     94.900   94.982243   -0.082243
4   140.0     89.925   89.998997   -0.073997


In [10]:
calls_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 780 entries, 0 to 1766
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   symbol             780 non-null    object         
 1   expiration         780 non-null    datetime64[ns] 
 2   optionType         780 non-null    object         
 3   contractSymbol     780 non-null    object         
 4   strike             780 non-null    float64        
 5   currency           780 non-null    object         
 6   lastPrice          780 non-null    float64        
 7   change             780 non-null    float64        
 8   percentChange      780 non-null    float64        
 9   volume             780 non-null    float64        
 10  openInterest       780 non-null    float64        
 11  bid                780 non-null    float64        
 12  ask                780 non-null    float64        
 13  contractSize       780 non-null    object         
 14

In [9]:

output_df = calls_df[['symbol', 'contractSymbol', 'expiration', 'days_to_expiry', 'strike', 
               'impliedVolatility', 'bid', 'ask', 'mid_price', 'bs_price', 'price_diff']]

output_df.to_csv('output.csv', index=False)



,symbol,contractSymbol,expiration,days_to_expiry,strike,impliedVolatility,bid,ask,mid_price,bs_price,price_diff
0,AAPL,AAPL241129C00100000,2024-11-29,6 days 02:57:10.225187,100.0,2.125005,129.20,130.60,129.900,129.956692,-0.056692
1,AAPL,AAPL241129C00115000,2024-11-29,6 days 02:57:10.225187,115.0,1.781251,114.15,115.65,114.900,114.967457,-0.067457
2,AAPL,AAPL241129C00130000,2024-11-29,6 days 02:57:10.225187,130.0,1.484378,99.20,100.60,99.900,99.978700,-0.078700
3,AAPL,AAPL241129C00135000,2024-11-29,6 days 02:57:10.225187,135.0,1.390628,94.15,95.65,94.900,94.982243,-0.082243
4,AAPL,AAPL241129C00140000,2024-11-29,6 days 02:57:10.225187,140.0,1.390628,89.25,90.60,89.925,89.998997,-0.073997
